In [1]:
import numpy as np
import serial
import datetime as dt
import os
from os.path import join, exists
import time
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Process, Queue, Event
import glob
import pandas as pd
import warnings
import sys

import subprocess
from tqdm.notebook import tqdm
import datetime

#if you want to display images as you record
import cv2
import matplotlib.pyplot as plt

from pyk4a import *

# imports from this module
from top_bottom_triggered.fast_animate import *
from top_bottom_triggered.commutator_utils import *
from top_bottom_triggered.video_io import *
from top_bottom_triggered.multicam_utils import *


# Known issues
* Keyboard interrupt doesn't work on the main acquisition loop. No idea why.
* If you prime the Azures, start the experiment, and then there's an error, in order to run the experiment again, you will need to:
    * manually unfreeze the azures by sending a short pulse of triggers and then interrupting them (see cell beneath the data collection cell). This is just a quirk of how the Azures work; when you prime them to wait for a trigger, they will block until they receive a trigger, no exceptions! (If you put a timeout, they raise C errors which are hard to catch in Python.)
    * re-create the Azure recording processes, and re-prime them (this is done for you).

# Experiment setup

## Get mouse order

In [2]:
# generate a random order for mice to run, based on today's date 
# (will be the same even, eg, 1 hour later, as long as date is the same)

mice_to_run = ['gmou77', 'gmou78', 'gmou81', 'gmou83']

today = dt.datetime.now().date()
date_hash = int(dt.datetime(today.year, today.month, today.day).timestamp())
np.random.seed(date_hash)
np.random.shuffle(mice_to_run)
print(mice_to_run)

['gmou77', 'gmou83', 'gmou81', 'gmou78']


## File name + dir, expt length

In [5]:
subject = 'test_minimal_example'
time_in_minutes = 0.4 # go slightly longer than mkv to ensure complete overlap
base_path = R'D:\Jonah\trigger_testing'
# base_path = R'E:\Jonah\CeAMouse'
file_suffix = ''  # disambiguate between two sessions on the same day
date = dt.datetime.now().strftime('%Y%m%d')

overwrite = True

In [6]:
path = os.path.join(base_path, f'{subject}\\{date}_{subject}')
# path = path.format(subject=subject, date=date)

if not os.path.exists(path):
    os.makedirs(path)
    print(f'Created {path}')
else:
    print(f'Path {path} exists!')

Path D:\Jonah\trigger_testing\test_minimal_example\20230523_test_minimal_example exists!


# Commutator Setup

In [7]:
commutator_port = 'COM4'
sync_device_port = 'COM7'

with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
    ino.write('r'.encode('utf-8')) ## reset trigger counter

In [8]:
show_opto = False  # only set to true if there is a "stim" col in ino data
debug = False

In [9]:
# test serial port and check dac value
with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
    line = ino.readline().decode('utf-8').strip('\r\n')
    print(line)
    print(f'Data has {len(line.split(","))} elements')

236633,117166,0
Data has 3 elements


In [10]:
commutator_fname = f'{date}_{subject}{file_suffix}.txt'
commutator_fullfile = os.path.join(path, commutator_fname)
if exists(commutator_fullfile) and not overwrite:
    raise ValueError(f'File {commutator_fullfile} exists! Add a suffix or change subject name.')
elif exists(commutator_fullfile):
    os.remove(commutator_fullfile)
else:
    pass

# Azure setup
Shouldn't really need to change this stuff

In [11]:
# 'bottom': '000343492012',  # old bottom
# 'bottom': '000693321712',  # new bottom

# 000364192012  # old top
# # new top
serial_numbers = {
    'bottom': '000693321712',
    'top': '000500221712'
}
master = 'top'  # don't change (should be top)
sync_delay,sync_delay_step = 0,500
record_processes = {}

file_prefix = os.path.join(path, f'{date}_{subject}' + file_suffix)
print(f'File will be saved to: {file_prefix}.XYZ')

File will be saved to: D:\Jonah\trigger_testing\test_minimal_example\20230523_test_minimal_example\20230523_test_minimal_example.XYZ


In [12]:
# If you get an error here, try unfreezing the azures; or just unplug + re-plug them. 
camera_indexes = get_camera_indexes(serial_numbers)
print(camera_indexes)

Index:0	Serial:000693321712	Color:1.6.110	Depth:1.6.80
Index:1	Serial:000500221712	Color:1.6.110	Depth:1.6.80
{'bottom': 0, 'top': 1}


## Prep the experiment!
(Three priming cells, and then the DAQ cell)

In [24]:
# Get the header from the arduino, and save it to the file
first_line = 1  # don't change
second_line = 0  # don't change
sync_sent = 0
header_max_attempts = 10
second_line_max_attempts = 10

with open(commutator_fullfile, 'x') as file:
    with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino:
        ino.write('r'.encode('utf-8')) ## reset trigger counter
        reader = ReadLine(ino)
        
        # These checks get header and process it
        if first_line:
            # Ask the arduino to print the header
            ino.write('h'.encode('utf-8'))

            # Verify first line. First_line becomes false when good (ie, we're no longer on the first line)
            status, first_line, second_line, header, n_attempts, read_lines = first_line_check(header_max_attempts, reader, file=file)
            if not status:
                raise RuntimeError('Didnt receive header!')
            
            trigger_idx = [i for i,val in enumerate(header.split(',')) if val=='trigger'][0]
            header_len = len(header.split(','))
            print(header)
        
        if second_line:
            status, second_line = second_line_check(second_line_max_attempts, reader, header, n_good_thresh=10)
        if not status:
                raise RuntimeError('Number of csv''d datapoints doesnt match number of csv''d elements in header!') 

time,data,trigger


In [25]:
# Set up azure processes

interrupt_queues = {camera: Queue() for camera,ix in camera_indexes.items()}
trigger_started_event = Event()
for camera,ix in camera_indexes.items():
    if camera==master:
        k4a = PyK4A(Config(color_resolution=ColorResolution.OFF,  # RES_720P
                           depth_mode=DepthMode.NFOV_UNBINNED,
                           synchronized_images_only=False,
                           wired_sync_mode=WiredSyncMode.SUBORDINATE), device_id=ix)
        
        p = Process(target=capture_from_azure, 
                    args=(k4a, file_prefix+'.'+camera, int(time_in_minutes*60)),
                    kwargs={
                        'display_time': True,
                        'display_frames':True,
                        'externally_triggered': True,
                        'trigger_started_event': trigger_started_event,
                        'interrupt_queue': interrupt_queues[camera]
                    })
        
    else:
        sync_delay += sync_delay_step
        k4a = PyK4A(Config(color_resolution=ColorResolution.OFF,
                           depth_mode=DepthMode.NFOV_UNBINNED,
                           synchronized_images_only=False,
                           wired_sync_mode=WiredSyncMode.SUBORDINATE,
                           subordinate_delay_off_master_usec=sync_delay), device_id=ix)

        p = Process(target=capture_from_azure, 
                    args=(k4a, file_prefix+'.'+camera, int(time_in_minutes*60)+5),
                    kwargs={
                        'display_time': camera==False,
                        'externally_triggered': True,
                        'interrupt_queue': interrupt_queues[camera]})

    record_processes[camera] = p
    
record_processes

{'bottom': <Process name='Process-5' parent=12868 initial>,
 'top': <Process name='Process-6' parent=12868 initial>}

In [26]:
# Start Azures   
for camera in camera_indexes:
    if camera != master:
        record_processes[camera].start()
time.sleep(3)
record_processes[master].start()
time.sleep(3)  # these sleep's are critical, st the Azure's are ready for the first trigger when it arrives
print('Azures primed...')

Azures primed...


#### Run this cell to start data acquisition!

In [27]:
# timing vars
start_time = dt.datetime.now()
one_mindelta = dt.timedelta(minutes=1)
exp_timedelta = time_in_minutes*one_mindelta # key var to be compared against (now - start_time)

# Main DAQ loop
try:
    with open(commutator_fullfile, 'a') as file:
        with serial.Serial(commutator_port, baudrate=115200, timeout=0.1) as ino, serial.Serial(sync_device_port, baudrate=9600, timeout=0.1) as sync_device:
            
            # More efficient serial reader
            reader = ReadLine(ino)
            
            while (dt.datetime.now() - start_time) < exp_timedelta:  
                
                # Read the line
                line = reader.readline().decode('utf-8').strip('\r\n')
                
                # Ensure trigger is starting at 0 (essential in order to align data later)
                if not (sync_sent):
                    assert int(line.split(',')[trigger_idx]) == 0
                
                # Remove the DEBUG output if present and debugging
                if debug:
                    line = line[:(line.find(',DEBUG:'))]    
                    
                # Assuming data looks good, start the sync device
                if not(sync_sent) and not(first_line or second_line):
                    print('sending start msg to sync device')
                    num = b"".join([packIntAsLong(int(time_in_minutes*60*30 + 300))])
                    sync_device.write(num)
                    sync_sent = 1
                    sync_response = sync_device.readline().decode("utf-8")
                    print(f'Sync device said: {sync_response}')
                    if 'pulses started!' in sync_response: 
                        print('Recording started successfully! Now we wait...')
                    else: raise RuntimeError('Got wrong response from sync device. Maybe try restarting it.')
                        
                # Check for the typical (but rare) serial read issues
                if len(line) == 0:
                    print('Got empty line, continuing...')
                    continue
                elif len(line.split(',')) != header_len:
                    print('Got line with unexpected length (skipping):')
                    print(line)
                    continue
                else:  
                    # typical case -- write line directly to file
                    file.write(line)
                    file.write('\n')
            
            # Join the Azure processes (ie block until they finish)
            if trigger_started_event.is_set():
                print('At end of recording. Joining az processes (may take a few moments...')
                exit_codes = [p.join() for p in record_processes.values() if p.is_alive()]

            print('Done with main loop')
            
# Catch unexpected errors            
except:
    print('Exception')
    # Stop the Azures in the event of an interrupt
    stop_azures(trigger_started_event, interrupt_queues, sync_device_port)
    
    raise
    
finally:
    print('Done.')

sending start msg to sync device
Sync device said: 1020 sync pulses started!

Recording started well! Now we wait...
joining az processes
Done with main loop
Done.


#### DEBUG
* "Cannot start process twice" --> re-run the cell where you create the Azure processes

#### DEBUG: Unfreeze Azures
Run this to send a short pulse of triggers via the syncing device to unfreeze the Azures, if necessary.

In [11]:
unfreeze_azures(sync_device_port)

'5 sync pulses started!\r\n'

#### DEBUG: Stop Azures
Run this to interrupt the Azures if they're running (eg because you unfroze them)

In [12]:
for q in interrupt_queues.values(): q.put(tuple())

NameError: name 'interrupt_queues' is not defined

## Post-experiment summaries

In [65]:
data = pd.read_csv(glob.glob(os.path.join(path, '*.txt'))[0])

In [66]:
therm_over_thresh_count = ((data.therm > 900) & (data.dac<=0.1)).sum()
therm_under_thresh_count = ((data.therm < 200) & (data.dac >= 3.25)).sum()
print(f'Therm over: {therm_over_thresh_count}')
print(f'Therm under: {therm_under_thresh_count}')
print(f'Time elapsed since start: {(dt.datetime.now() - start_time).seconds/60:0.1f} minutes')

Therm over: 4608
Therm under: 0
Time elapsed since start: 66.8 minutes
